# Analysis of AI‑Generated News Articles for Linguistic Strategy Detection

This notebook runs SHAP (SHapley Additive exPlanations) analysis on LightGBM models trained in "Propaganda by Prompt: Tracing Hidden Linguistic Strategies in Large Language Models." 

## Table of Contents
1. [Environment Setup](#1-environment-setup)
2. [Model Configuration](#2-model-configuration)
3. [Data Preparation](#3-data-preparation)
4. [Model Performance Analysis](#4-model-performance-analysis)
5. [SHAP Analysis](#5-shap-analysis)
   - [Summary Plots](#summary-plots)
   - [Feature Dependence Analysis](#feature-dependence-analysis)

## 1. Environment Setup

This notebook requires several Python packages for machine learning, data analysis, and visualization. Install dependencies using:

```bash
pip install -r requirements.txt
```

In [ ]:
import pickle
import shap
import pandas as pd
import sqlite3
import matplotlib.pyplot as plt
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    roc_auc_score, roc_curve, auc, precision_recall_curve
)
from sklearn import model_selection
import numpy as np

## 2. Model Configuration

The notebook supports analysis of multiple LightGBM models trained on different subsets of data:

- **Combined Models**: Analyze all topics together
- **Topic-specific Models**: Separate models for climate change, COVID-19, the Capitol riot, and LGBT topics
- **Model Variants**: 
  - Stored in the models directory
  - With/without punctuation features
  - Different GPT versions (GPT-3.5, GPT-4o, GPT-4.1)

Each model configuration includes:
- SQL query for data selection
- Target variable specification
- Feature set configuration (with/without punctuation)

In [ ]:
# Define your configuration dictionary
model_configs = {
    "models/LGBM_Combined_DetectHumanProp.pkl": {
        "query": 'SELECT * FROM ai_prop_liwc WHERE WC >= 100 AND model LIKE "human"',
        "target_variable": "Label",
        "x_features": "withPunc"
    },
    "models/LGBM_Combined_DetectAI_GPT35_noPunc.pkl": {
        "query": 'SELECT * FROM ai_prop_liwc WHERE WC >= 100 AND model IN ("gpt-3.5-turbo", "human")',
        "target_variable": "AI",
        "x_features": "noPunc"
    },
    "models/LGBM_Combined_DetectAIProp_GPT35.pkl": {
        "query": 'SELECT * FROM ai_prop_liwc WHERE WC >= 100 AND model LIKE "gpt-3.5-turbo"',
        "target_variable": "Label",
        "x_features": "withPunc"
    },
    "models/LGBM_Combined_DetectAIProp_GPT35_noPunc.pkl": {
        "query": 'SELECT * FROM ai_prop_liwc WHERE WC >= 100 AND model LIKE "gpt-3.5-turbo"',
        "target_variable": "Label",
        "x_features": "noPunc"
    },
    "models/LGBM_Combined_DetectAIProp_GPT4o.pkl": {
        "query": 'SELECT * FROM ai_prop_liwc WHERE WC >= 100 AND model LIKE "gpt-4o"',
        "target_variable": "Label",
        "x_features": "withPunc"
    },
    "models/LGBM_Combined_DetectAIProp_GPT4o_noPunc.pkl": {
        "query": 'SELECT * FROM ai_prop_liwc WHERE WC >= 100 AND model LIKE "gpt-4o"',
        "target_variable": "Label",
        "x_features": "noPunc"
    },
    "models/LGBM_Combined_DetectAIProp_GPT41.pkl": {
        "query": 'SELECT * FROM ai_prop_liwc WHERE WC >= 100 AND model LIKE "gpt-4.1"',
        "target_variable": "Label",
        "x_features": "withPunc"
    },
    "models/LGBM_Combined_DetectAIProp_GPT41_noPunc.pkl": {
        "query": 'SELECT * FROM ai_prop_liwc WHERE WC >= 100 AND model LIKE "gpt-4.1"',
        "target_variable": "Label",
        "x_features": "noPunc"
    },
    "models/LGBM_climate_DetectAIProp_GPT35.pkl": {
        "query": 'SELECT * FROM ai_prop_liwc WHERE WC >= 100 AND model LIKE "gpt-3.5-turbo" AND topic LIKE "climate"',
        "target_variable": "Label",
        "x_features": "withPunc"
    },
    "models/LGBM_climate_DetectAIProp_GPT35_noPunc.pkl": {
        "query": 'SELECT * FROM ai_prop_liwc WHERE WC >= 100 AND model LIKE "gpt-3.5-turbo" AND topic LIKE "climate"',
        "target_variable": "Label",
        "x_features": "noPunc"
    },
    "models/LGBM_climate_DetectAIProp_GPT4o.pkl": {
        "query": 'SELECT * FROM ai_prop_liwc WHERE WC >= 100 AND model LIKE "gpt-4o" AND topic LIKE "climate"',
        "target_variable": "Label",
        "x_features": "withPunc"
    },
    "models/LGBM_climate_DetectAIProp_GPT4o_noPunc.pkl": {
        "query": 'SELECT * FROM ai_prop_liwc WHERE WC >= 100 AND model LIKE "gpt-4o" AND topic LIKE "climate"',
        "target_variable": "Label",
        "x_features": "noPunc"
    },
    "models/LGBM_climate_DetectAIProp_GPT41.pkl": {
        "query": 'SELECT * FROM ai_prop_liwc WHERE WC >= 100 AND model LIKE "gpt-4.1" AND topic LIKE "climate"',
        "target_variable": "Label",
        "x_features": "withPunc"
    },
    "models/LGBM_climate_DetectAIProp_GPT41_noPunc.pkl": {
        "query": 'SELECT * FROM ai_prop_liwc WHERE WC >= 100 AND model LIKE "gpt-4.1" AND topic LIKE "climate"',
        "target_variable": "Label",
        "x_features": "noPunc"
    },
    "models/LGBM_covid_DetectAIProp_GPT35.pkl": {
        "query": 'SELECT * FROM ai_prop_liwc WHERE WC >= 100 AND model LIKE "gpt-3.5-turbo" AND topic LIKE "covid"',
        "target_variable": "Label",
        "x_features": "withPunc"
    },
    "models/LGBM_covid_DetectAIProp_GPT35_noPunc.pkl": {
        "query": 'SELECT * FROM ai_prop_liwc WHERE WC >= 100 AND model LIKE "gpt-3.5-turbo" AND topic LIKE "covid"',
        "target_variable": "Label",
        "x_features": "noPunc"
    },
    "models/LGBM_covid_DetectAIProp_GPT4o.pkl": {
        "query": 'SELECT * FROM ai_prop_liwc WHERE WC >= 100 AND model LIKE "gpt-4o" AND topic LIKE "covid"',
        "target_variable": "Label",
        "x_features": "withPunc"
    },
    "models/LGBM_covid_DetectAIProp_GPT4o_noPunc.pkl": {
        "query": 'SELECT * FROM ai_prop_liwc WHERE WC >= 100 AND model LIKE "gpt-4o" AND topic LIKE "covid"',
        "target_variable": "Label",
        "x_features": "noPunc"
    },
    "models/LGBM_covid_DetectAIProp_GPT41.pkl": {
        "query": 'SELECT * FROM ai_prop_liwc WHERE WC >= 100 AND model LIKE "gpt-4.1" AND topic LIKE "covid"',
        "target_variable": "Label",
        "x_features": "withPunc"
    },
    "models/LGBM_covid_DetectAIProp_GPT41_noPunc.pkl": {
        "query": 'SELECT * FROM ai_prop_liwc WHERE WC >= 100 AND model LIKE "gpt-4.1" AND topic LIKE "covid"',
        "target_variable": "Label",
        "x_features": "noPunc"
    },
    "models/LGBM_capitolriot_DetectAIProp_GPT35.pkl": {
        "query": 'SELECT * FROM ai_prop_liwc WHERE WC >= 100 AND model LIKE "gpt-3.5-turbo" AND topic LIKE "capitolriot"',
        "target_variable": "Label",
        "x_features": "withPunc"
    },
    "models/LGBM_capitolriot_DetectAIProp_GPT35_noPunc.pkl": {
        "query": 'SELECT * FROM ai_prop_liwc WHERE WC >= 100 AND model LIKE "gpt-3.5-turbo" AND topic LIKE "capitolriot"',
        "target_variable": "Label",
        "x_features": "noPunc"
    },
    "models/LGBM_capitolriot_DetectAIProp_GPT4o.pkl": {
        "query": 'SELECT * FROM ai_prop_liwc WHERE WC >= 100 AND model LIKE "gpt-4o" AND topic LIKE "capitolriot"',
        "target_variable": "Label",
        "x_features": "withPunc"
    },
    "models/LGBM_capitolriot_DetectAIProp_GPT4o_noPunc.pkl": {
        "query": 'SELECT * FROM ai_prop_liwc WHERE WC >= 100 AND model LIKE "gpt-4o" AND topic LIKE "capitolriot"',
        "target_variable": "Label",
        "x_features": "noPunc"
    },
    "models/LGBM_capitolriot_DetectAIProp_GPT41.pkl": {
        "query": 'SELECT * FROM ai_prop_liwc WHERE WC >= 100 AND model LIKE "gpt-4.1" AND topic LIKE "capitolriot"',
        "target_variable": "Label",
        "x_features": "withPunc"
    },
    "models/LGBM_capitolriot_DetectAIProp_GPT41_noPunc.pkl": {
        "query": 'SELECT * FROM ai_prop_liwc WHERE WC >= 100 AND model LIKE "gpt-4.1" AND topic LIKE "capitolriot"',
        "target_variable": "Label",
        "x_features": "noPunc"
    },
    "models/LGBM_lgbt_DetectAIProp_GPT35.pkl": {
        "query": 'SELECT * FROM ai_prop_liwc WHERE WC >= 100 AND model LIKE "gpt-3.5-turbo" AND topic LIKE "lgbt"',
        "target_variable": "Label",
        "x_features": "withPunc"
    },
    "models/LGBM_lgbt_DetectAIProp_GPT35_noPunc.pkl": {
        "query": 'SELECT * FROM ai_prop_liwc WHERE WC >= 100 AND model LIKE "gpt-3.5-turbo" AND topic LIKE "lgbt"',
        "target_variable": "Label",
        "x_features": "noPunc"
    },
    "models/LGBM_lgbt_DetectAIProp_GPT4o.pkl": {
        "query": 'SELECT * FROM ai_prop_liwc WHERE WC >= 100 AND model LIKE "gpt-4o" AND topic LIKE "lgbt"',
        "target_variable": "Label",
        "x_features": "withPunc"
    },
    "models/LGBM_lgbt_DetectAIProp_GPT4o_noPunc.pkl": {
        "query": 'SELECT * FROM ai_prop_liwc WHERE WC >= 100 AND model LIKE "gpt-4o" AND topic LIKE "lgbt"',
        "target_variable": "Label",
        "x_features": "noPunc"
    },
    "models/LGBM_lgbt_DetectAIProp_GPT41.pkl": {
        "query": 'SELECT * FROM ai_prop_liwc WHERE WC >= 100 AND model LIKE "gpt-4.1" AND topic LIKE "lgbt"',
        "target_variable": "Label",
        "x_features": "withPunc"
    },
    "models/LGBM_lgbt_DetectAIProp_GPT41_noPunc.pkl": {
        "query": 'SELECT * FROM ai_prop_liwc WHERE WC >= 100 AND model LIKE "gpt-4.1" AND topic LIKE "lgbt"',
        "target_variable": "Label",
        "x_features": "noPunc"
    },
}

print(len(model_configs), "models loaded.")

In [ ]:
# Choose the model you want to analyze by selecting a model path from model_configs
model_path = 'models/LGBM_Combined_DetectAIProp_GPT41_noPunc.pkl'  
query = model_configs[model_path]['query']
target_variable = model_configs[model_path]['target_variable']  
x_features = model_configs[model_path]['x_features']

In [ ]:
# Execute query and load results into pandas dataframe
def execute_query_pandas(path, query):
    """Execute SQL query and return results as pandas DataFrame."""
    conn = sqlite3.connect(path)
    df = pd.read_sql_query(query, conn)
    return df

# Load the model from the .pkl file
with open(model_path, 'rb') as file:
    model = pickle.load(file)

## 3. Data Preparation

The data preparation process involves several steps:

1. **Database Query**: Select relevant articles based on:
   - Minimum word count (WC ≥ 100)
   - Model type (human/AI)
   - Topic (if applicable)

2. **Feature Processing**:
   - Remove non-LIWC features (metadata, text content)
   - Optional removal of punctuation features
   - Feature normalization to [0,1] range

3. **Data Split**:
   - 70% training / 30% testing
   - Stratified sampling to maintain class distribution

In [ ]:
# Load your dataset
path = f'ai_propaganda.db' #Assign the path to the database (assumes its in the same directory)

data = execute_query_pandas(path,query)

# The features to remove are selected based on the chosen model (with or without punctuation features)
if x_features == "withPunc":
    feature_columns = [
                "Analytic", "Clout", "Authentic", "Tone", "WPS", "BigWords", "Dic", "Linguistic",
                "function", "pronoun", "ppron", "i", "we", "you", "shehe", "they", "ipron", "det",
                "article", "number", "prep", "auxverb", "adverb", "conj", "negate", "verb", "adj",
                "quantity", "Drives", "affiliation", "achieve", "power", "Cognition", "allnone",
                "cogproc", "insight", "cause", "discrep", "tentat", "certitude", "differ", "memory",
                "Affect", "tone_pos", "tone_neg", "emotion", "emo_pos", "emo_neg", "emo_anx",
                "emo_anger", "emo_sad", "swear", "Social", "socbehav", "prosocial", "polite", "conflict",
                "moral", "comm", "socrefs", "family", "friend", "female", "male", "Culture", "politic",
                "ethnicity", "tech", "Lifestyle", "leisure", "home", "work", "money", "relig",
                "Physical", "health", "illness", "wellness", "mental", "substances", "sexual", "food",
                "death", "need", "want", "acquire", "lack", "fulfill", "fatigue", "reward", "risk",
                "curiosity", "allure", "Perception", "attention", "motion", "space", "visual",
                "auditory", "feeling", "time", "focuspast", "focuspresent", "focusfuture",
                "Conversation", "netspeak", "assent", "nonflu", "filler", "AllPunc", "Period", "Comma",
                "QMark", "Exclam", "Apostro", "OtherP", "Emoji"
            ]
elif x_features == "noPunc":
    feature_columns = [
                "Analytic", "Clout", "Authentic", "Tone", "WPS", "BigWords", "Dic", "Linguistic",
                "function", "pronoun", "ppron", "i", "we", "you", "shehe", "they", "ipron", "det",
                "article", "number", "prep", "auxverb", "adverb", "conj", "negate", "verb", "adj",
                "quantity", "Drives", "affiliation", "achieve", "power", "Cognition", "allnone",
                "cogproc", "insight", "cause", "discrep", "tentat", "certitude", "differ", "memory",
                "Affect", "tone_pos", "tone_neg", "emotion", "emo_pos", "emo_neg", "emo_anx",
                "emo_anger", "emo_sad", "swear", "Social", "socbehav", "prosocial", "polite", "conflict",
                "moral", "comm", "socrefs", "family", "friend", "female", "male", "Culture", "politic",
                "ethnicity", "tech", "Lifestyle", "leisure", "home", "work", "money", "relig",
                "Physical", "health", "illness", "wellness", "mental", "substances", "sexual", "food",
                "death", "need", "want", "acquire", "lack", "fulfill", "fatigue", "reward", "risk",
                "curiosity", "allure", "Perception", "attention", "motion", "space", "visual",
                "auditory", "feeling", "time", "focuspast", "focuspresent", "focusfuture",
                "Conversation", "netspeak", "assent", "nonflu", "filler", "Emoji"
            ]
else:
    raise ValueError("Invalid x_features value. Please ensure you have selected a valid model path.")

X = data[feature_columns].copy()
# X = data.drop(remove_feat_list, axis=1)
y = data[target_variable]

# Normalize feature values to range [0, 1]
for column in X.columns:
        #print(f'{column} has max of {X[column].max()}')
        if X[column].max() == 0: #Check for columns that contain no LIWC values 
            print(f'Column {column} can be removed.')
        X[column] = (X[column] - X[column].min()) / (X[column].max() - X[column].min())

# Split data
test_size = 0.3
X_train, X_test, y_train, y_test = model_selection.train_test_split(
    X, y, test_size=test_size, stratify=y, random_state=42
)

## 4. Model Performance Analysis

The analysis includes comprehensive classification metrics:

1. **Basic Metrics**:
   - Accuracy: Overall prediction accuracy
   - Precision: Accuracy of positive predictions
   - Recall: Proportion of actual positives identified
   - F1 Score: Harmonic mean of precision and recall

2. **Advanced Metrics**:
   - ROC-AUC: Area under ROC curve
   - PR-AUC: Area under Precision-Recall curve
   - Confusion Matrix Components

3. **Class Distribution**:
   - Prediction Rate: Proportion of positive predictions
   - Positive Class Rate: Actual proportion of positive cases

In [ ]:
def get_classification_metrics(y_test: np.ndarray, y_predicted: np.ndarray, prob: np.ndarray) -> pd.DataFrame:
    """
    Calculate comprehensive classification metrics for model evaluation.
    
    Args:
        y_test: Array of true labels
        y_predicted: Array of predicted labels
        prob: Array of prediction probabilities
        
    Returns:
        DataFrame containing various classification metrics:
        - Basic metrics: Accuracy, Precision, Recall, F1
        - Advanced metrics: ROC-AUC, PR-AUC
        - Class distribution metrics: Prediction rates
        - Confusion matrix components: TP, TN, FP, FN
    """
    # Basic metrics
    accuracy = round(accuracy_score(y_test, y_predicted), 2)
    precision = round(precision_score(y_test, y_predicted), 2)
    precision_0 = round(precision_score(y_test, y_predicted, pos_label=0), 2)
    recall = round(recall_score(y_test, y_predicted), 2)
    specificity = round(recall_score(y_test, y_predicted, pos_label=0), 2)
    f1 = round(f1_score(y_test, y_predicted), 2)
    f1_0 = round(f1_score(y_test, y_predicted, pos_label=0), 2)
    
    # AUC metrics
    p, r, th = precision_recall_curve(y_test, prob)
    pr_auc = round(auc(r, p), 2)
    
    if len(np.unique(y_test)) == 1:
        roc = np.NaN
    else:
        roc = round(roc_auc_score(y_test, prob), 2)
    
    # Rates
    pos_pred_rate = round(sum(y_predicted) * 100 / len(y_predicted), 2)
    pos_rate = round(sum(y_test) * 100 / len(y_test), 2)
    
    # Confusion matrix components
    temp = pd.DataFrame({'actual': y_test, 'prediction': y_predicted})
    tp = len(temp[(temp['actual'] == 1) & (temp['prediction'] == 1)])
    tn = len(temp[(temp['actual'] == 0) & (temp['prediction'] == 0)])
    fp = len(temp[(temp['actual'] == 0) & (temp['prediction'] == 1)])
    fn = len(temp[(temp['actual'] == 1) & (temp['prediction'] == 0)])
    
    metrics = {
        'metrics': [
            "Accuracy", "Precision", "Recall", "Specificity", "Precision_0",
            "F1", "F1_0", "PR_AUC", "ROC", "TP", "FP", "TN", "FN",
            "PredictionRate", "PositiveClassRate", "Count"
        ],
        'value': [
            accuracy, precision, recall, specificity, precision_0,
            f1, f1_0, pr_auc, roc, tp, fp, tn, fn,
            pos_pred_rate, pos_rate, len(y_test)
        ]
    }
    
    return pd.DataFrame(metrics)

def find_optimal_threshold(y_test: np.ndarray, y_pred_prob: np.ndarray) -> float:
    """
    Find optimal classification threshold using Youden's J statistic
    
    Args:
        y_test: True labels
        y_pred_prob: Prediction probabilities
        
    Returns:
        Optimal threshold value
    """
    fpr, tpr, thresholds = roc_curve(y_test, y_pred_prob)
    threshold = thresholds[np.argmax(tpr - fpr)]
    return threshold


# Get model predictions
y_pred_prob = model.predict(X_test)
threshold = find_optimal_threshold(y_test, y_pred_prob)
y_pred = (y_pred_prob >= threshold).astype(int)

# Calculate and display metrics
metrics_df = get_classification_metrics(y_test, y_pred, y_pred_prob)
print("\nModel Performance Metrics:")
print("="*50)
print(metrics_df)

## 5. SHAP Analysis

SHAP (SHapley Additive exPlanations) analysis reveals how each feature contributes to the model's predictions.

To save plots, uncomment the `plt.savefig()` lines


In [ ]:
# Initialize the SHAP explainer
explainer = shap.TreeExplainer(model)

# Calculate SHAP values
shap_values = explainer.shap_values(X)

# Enables interactive plots to open in a separate window, which allows for zooming, panning, and resizing.
# Requires the PyQt6 package to be installed as the backend for rendering the window.
%matplotlib qt


### Summary Plots

1. **Beeswarm Plot**:
   - Shows feature importance distribution
   - Red = high feature values
   - Blue = low feature values
   - Horizontal spread = SHAP value impact

2. **Bar Plot**:
   - Average magnitude of feature importance
   - Helps identify top contributing features

In [ ]:
shap.initjs()
shap.summary_plot(shap_values, X, max_display=10)

# Get the current figure and axes objects. from @GarrettCGraham code
fig, ax = plt.gcf(), plt.gca()

# Modifying main plot parameters
ax.tick_params(labelsize=12)
ax.set_xlabel("SHAP value (impact on model output)", fontsize=12)

# Get colorbar
cb_ax = fig.axes[1] 

# Modifying color bar parameters
cb_ax.tick_params(labelsize=14)
cb_ax.set_ylabel("Feature value", fontsize=12)

ax.yaxis.grid(linestyle='--', linewidth='.6', color='grey')

ax.tick_params(axis='x', colors='black')
ax.tick_params(axis='y', colors='black')

# Uncomment below to save the plot
# plt.savefig('SHAP_Top_10_Beeswarm.png',format = "png",dpi = 300,bbox_inches = 'tight')

In [ ]:
shap_plot = shap.summary_plot(shap_values, X, plot_type="bar", color='#9fb2d1',max_display=10)
ax = plt.gca()
ax.tick_params(labelsize=12)

ax.xaxis.label.set_fontsize(12)
ax.yaxis.grid(linestyle='--', linewidth='0.6', color='grey')
ax.tick_params(axis='y', labelsize=16, colors='black')  # Increase Y-axis label size

# Uncomment below to save the plot
# plt.savefig('SHAP_Top_10_Barplot.png',format = "png",dpi = 300,bbox_inches = 'tight')

### Feature Dependence Analysis

Examines relationships between:
- Individual feature values and their SHAP values
- Interactions between feature pairs
- Specific combinations of linguistic features



In [ ]:
# Optionally, create a SHAP dependence plot for specific features
features = ["WPS", "BigWords"]

for feature_name in features:
    dep_plot = shap.dependence_plot(feature_name, shap_values, X) #Automatically chooses interaction
    #shap.dependence_plot('work', shap_values, X, interaction_index='we')

    ax = plt.gca()
    ax.xaxis.label.set_fontsize(13)
    ax.yaxis.label.set_fontsize(13)

    cbar = plt.gcf().get_axes()[-1]  # Get the color bar axis
    print("Color bar label font size:", cbar.yaxis.label.get_size())

    # Uncomment below to save the plot
    # plt.savefig(f'SHAP_{feature_name}_Dependence.png',format = "png",dpi = 300,bbox_inches = 'tight')
    

In [ ]:
# Specific Dependence Plot Combos
combos = [("Cognition","Analytic"), ("we","moral")]

for combo in combos:
    dep_plot = shap.dependence_plot(combo[0], shap_values, X, interaction_index=combo[1])
    ax = plt.gca()
    ax.xaxis.label.set_fontsize(13)
    ax.yaxis.label.set_fontsize(13)

    cbar = plt.gcf().get_axes()[-1]  # Get the color bar axis
    #print("Color bar label font size:", cbar.yaxis.label.get_size())

    # Uncomment below to save the plot
    # plt.savefig(f'SHAP_{combo[0]}_and_{combo[1]}_Dependence.png',format = "png",dpi = 300,bbox_inches = 'tight')